Product Name	Inventory Number/SKU	UPC/EAN	ASIN	MPN	Product Price	Cost	Labels	MSRP	Category	MAP

Product Name	Inventory Number	UPC/EAN	MPN	Product Price	Cost	New Price	Condition	SUBCATEGORY	UPC	VENDNAME	Category	MAP


In [41]:
%%time

import warnings

warnings.filterwarnings('ignore')

from functools import reduce

import numpy as np

import pyodbc

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

import pandas as pd

import sys


#####
#AX items
#####

query = '''

SELECT *

FROM SD_items_by_vendor_UPC2

WHERE Discontinued = ''

'''

dfLHA = pd.read_sql(query, cnxn)


#####
#bar codes
#####

query = '''

SELECT

    ITEMID,
    ITEMBARCODE

FROM SD_Latest_BarCode

'''

dfBarCode = pd.read_sql(query, cnxn)


#####
#cost
#####

query = '''

SELECT *

FROM SD_LANDED_COST

'''

dfLandedCost = pd.read_sql(query, cnxn)


query = '''

SELECT *

FROM SD_Standard_Cost

'''

dfStandardCost = pd.read_sql(query, cnxn)


#####
#web price
#####

query = '''

SELECT *

FROM SD_web_pricing

'''

dfWebPrice = pd.read_sql(query, cnxn)


#####
#retail price
#####

query = '''

SELECT *

FROM SD_Current_Retail_Price

'''

dfRetailPrice = pd.read_sql(query, cnxn)

#####
#categories
#####

query = '''

SELECT *

FROM SD_AX_Item_Categories

'''

dfCategories = pd.read_sql(query, cnxn)


#####
#MAP
#####

query = '''

SELECT 

    ITEMID,
    LHAMAPPRICE AS 'MAP'

FROM InventTable

'''

dfMAP = pd.read_sql(query, cnxn)

try:
    dfRank = pd.read_excel(r'V:\Jason\PopularityRankTable.xlsx', sheet_name='Popularity')
    dfRank.rename(columns={'ItemID': 'ITEMID', 'Overall': 'Rank'}, inplace=True)
    # print('Ready.')
    
except:
    print('User has file open elsewhere.')
    sys.exit(0)
    
#shrink tables before merging
dfLHA.drop(dfLHA.columns[[2,5,6,7,8,9,10,11,12,13,15]], axis = 1, inplace = True)
dfLHA.rename(columns={'NAMEALIAS': 'Product Name', 'ExternalID': 'MPN', 'Vendor Name': 'VendName'}, inplace=True)

dfBarCode.rename(columns={'ITEMBARCODE': 'UPC/EAN'}, inplace=True)

dfCategories.drop(dfCategories.columns[[3,4]], axis = 1, inplace = True)

dfRetailPrice.drop(dfRetailPrice.columns[[2,3]], axis = 1, inplace = True)
dfRetailPrice.rename(columns={'PRICE': 'RETAIL'}, inplace=True)

dfWebPrice.drop(dfWebPrice.columns[[1,2,3,4,6,7]], axis = 1, inplace = True)
dfWebPrice = dfWebPrice.groupby('ITEMID', as_index=False).agg(min)

dfRank.drop(dfRank.columns[[1,2,4]], axis = 1, inplace = True)

dfLandedCost.drop(dfLandedCost.columns[[1,2,3,4,5,6,7,9]], axis = 1, inplace = True)

dfStandardCost.drop(dfStandardCost.columns[[2,3,4]], axis = 1, inplace = True)
dfStandardCost.rename(columns={'PRICE': 'StdCost'}, inplace=True)

# conditional column cost

data_frames = [dfLandedCost, dfStandardCost]

df_cost_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='outer'), data_frames).fillna('')

df_cost_merged['Cost'] = np.where((df_cost_merged['LandedCost'] == 'null'), df_cost_merged['StdCost'], df_cost_merged['LandedCost'])

df_cost_merged.drop(['LandedCost', 'StdCost'], axis = 1, inplace = True)

# conditional column retail price

data_frames = [dfRetailPrice, dfWebPrice]

df_price_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='outer'), data_frames).fillna(0)

# conditions = [
#     (df_price_merged['MAP'] != 0) & (df_price_merged['MAP'] < df_price_merged['WEBPRICE']), #use MAP if it is not 0 and less than the web price
#     (df_price_merged['WEBPRICE'] == 0) & (df_price_merged['MAP'] == 0), #if the web price and MAP are 0, use retail
#     ]

# choices = [df_price_merged['MAP'], df_price_merged['PRICE']]

# df_price_merged['Product Price'] = np.select(conditions, choices, default=df_price_merged['WEBPRICE'])

df_price_merged['Product Price'] = np.where((df_price_merged['WEBPRICE'] == 0), df_price_merged['RETAIL'], df_price_merged['WEBPRICE'])

df_price_merged.drop(['RETAIL', 'WEBPRICE'], axis = 1, inplace = True)

df_price_merged['Product Price'] = df_price_merged['Product Price'].round(2)

# compile the list of dataframes you want to merge
data_frames = [dfLHA, dfCategories, df_cost_merged, df_price_merged, dfMAP, dfRank, dfBarCode]

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='left'), data_frames).fillna('')

df_merged = df_merged.reindex(columns=['Product Name', 'ITEMID', 'UPC/EAN','MPN','Product Price', 
                                       'Cost','Category', 'SubCategory', 'MAP', 'Published', 'Rank'])

#drop all rows w/o a UPC
df_merged['UPC/EAN'].replace('', np.nan, inplace=True)
df_merged.dropna(subset=['UPC/EAN'], inplace=True)

#drop duplicates based on SKU and UPC
df_merged.drop_duplicates(subset = ['ITEMID', 'UPC/EAN'], keep = 'first', inplace=True)

df_merged.reset_index(drop=True, inplace=True)

maxRows = 25000

df_merged.sort_values(by=['Rank'])

df_merged_capped = df_merged.head(maxRows)

df_merged_capped.drop(['Published', 'Rank'], axis = 1, inplace = True)

df_merged_capped.sort_values(by=['ITEMID'])

print('Ready.')

Ready.
CPU times: total: 17 s
Wall time: 23 s


In [43]:
df_merged_capped.to_excel('2022_08_17 Wiser Upload FULL.xlsx', index=False, header=True)

In [42]:
numOfRows = len(df_merged_capped.index)

print('Total number of rows in df_merged_capped: ' , numOfRows)

Total number of rows in df_merged_capped:  25000


In [12]:
# dfCheck = df_merged[df_merged['ITEMID'] == '2499']

# dfCheck.head()

,Product Name,ITEMID,UPC/EAN,MPN,Product Price,Cost,Category,SubCategory,MAP,Published,Rank
21154,Goat's Milk Soap 5oz Bar,2499,779242099249,9924,3.95,3.84,Home Indoors,Personal Products,0.0,0,A


In [ ]:
# df_duplicatesFIRST = df_merged.loc[df_merged['ITEMID'].duplicated(keep='first')]

# df_duplicatesLAST = df_merged.loc[df_merged['ITEMID'].duplicated(keep='last')]

# df_duplicates = pd.concat([df_duplicatesFIRST, df_duplicatesLAST])

# df_duplicates.drop_duplicates(subset = ['UPC/EAN'], keep = False, inplace = True)

# df_duplicates.reset_index(drop=True, inplace=True)

# df_duplicates.sort_values('ITEMID')

# df_duplicates.to_excel('2022_08_16 Wiser Upload Duplicates.xlsx', index=False, header=True)

# df_merged.drop_duplicates(subset = ['UPC/EAN'], keep = 'first', inplace = True)

# df_merged.reset_index(drop=True, inplace=True)

# df_merged.sort_values('ITEMID')

# df_merged.to_excel('2022_08_17 Wiser Upload FULL.xlsx', index=False, header=True)

# df_merged.head()

In [32]:
numOfRows = len(df_merged.index)
print('Total number of rows in df_merged: ' , numOfRows)

# count of unique SKUs in the dataframe
uniqueMatches = df_merged['UPC/EAN'].nunique()

print('Number of unique rows in df_merged by UPC/EAN: ', uniqueMatches)
print('*******************************')

df_merged_filtered = df_merged[df_merged['Rank'] != 'X']

numOfRows = len(df_merged_filtered.index)
print('Total number of rows in df_merged_filtered by Rank not X: ' , numOfRows)

df_merged_filtered = df_merged

df_merged_filtered = df_merged[df_merged['Rank'] != 'D']

numOfRows = len(df_merged_filtered.index)
print('Total number of rows in df_merged_filtered by Rank not D: ' , numOfRows)

df_merged_filtered = df_merged

df_merged_filtered = df_merged[df_merged['Product Price'] >= 10]

numOfRows = len(df_merged_filtered.index)
print('Total number of rows in df_merged_filtered by Product Price more than $10: ' , numOfRows)

Total number of rows in df_merged:  25459
Number of unique rows in df_merged by UPC/EAN:  25459
*******************************
Total number of rows in df_merged_filtered by Rank not X:  21849
Total number of rows in df_merged_filtered by Rank not D:  14568
Total number of rows in df_merged_filtered by Product Price more than $10:  16534


In [31]:
df_merged.head()

,Product Name,ITEMID,UPC/EAN,MPN,Product Price,Cost,Category,SubCategory,MAP,Published,Rank
0,Wick for Gimbaled Berth Lamp,00006W,8715221008682,00006W,14.99,8.23,Lighting,Flame,0.0,0,D
1,Jelly 9oz Elderberry,0010010,815310010010,,4.95,1.82,Food,Canned Food,0.0,0,C
2,Jam 9oz Blackraspberry Seedless,0010089,815310010089,,4.95,3.04,Food,Canned Food,0.0,0,D
3,Opal Ribbed Shade 7in,00616,810020125108,00616,47.99,21.3,Lighting,Flame,47.2,1,D
4,Bag Balm 1oz,01935,098193000051,176-397,8.99,4.75,Home Indoors,Personal Products,0.0,0,B


In [97]:
# # dropping ALL duplicate values
# df_merged.drop_duplicates(subset = ['ITEMID'], keep = False, inplace = True)

# # count of unique SKUs in the dataframe
# uniqueMatches = df_merged['ITEMID'].nunique()

# print('Number of unique rows in dataframe by SKU: ', uniqueMatches)

Number of unique rows in dataframe by SKU:  23974


In [91]:
myColumnCount = len(dfLHA.columns)

i = 0

while i < myColumnCount:
    col = dfLHA.columns[i]
    # Get column index position of column
    col_index = list(dfLHA.columns).index(dfLHA.columns[i])
    print(str(col_index) + ' - ' + col)
    i += 1

0 - ITEMID
1 - NAMEALIAS
2 - PrimaryVendorId
3 - Vendor Name
4 - ExternalID
5 - BarCode
6 - BARCODESETUPID
7 - Buyer
8 - ParentSKU
9 - Ex-stock
10 - ManualCoverage
11 - Discontinued
12 - InventoryAvailable
13 - BackOrdered
14 - Published
15 - SyncCA


In [92]:
#####
#bar codes
#####

query = '''

SELECT

    ITEMID,
    ITEMBARCODE

FROM SD_Latest_BarCode

'''

dfBarCode = pd.read_sql(query, cnxn)

In [93]:
dfBarCode.head()

,ITEMID,ITEMBARCODE
0,100027136,841857113754
1,100065508,840224233606
2,100056568,074936000137
3,100061292,738215542854
4,100037080,696527251638
